In [1]:
import re
import asyncio
import os
from simbricks.orchestration import system as sys_mod
from simbricks.orchestration import simulation
from simbricks.orchestration import instantiation
from simbricks.utils import base as utils_base
from simbricks.client.opus import base as opus_base
from simbricks.orchestration.helpers import simulation as sim_helpers
from simbricks.orchestration.helpers import instantiation as inst_helpers

# Lab 1: Configuring & Running Virtual Prototypes with SimBricks Cloud

## 1. Minimal 2-Host Virtual Prototype
To get started, we will set up a simple virtual prototype that is composed of two hosts that have one NIC each and are connected through a basic switch. The hosts run Linux with the regular network stack, and run the standard `iperf` network benchmark, with one host acting as the server, and the other as the client.

To keep simulation times low for testing here, we use unsynchronized and inaccurate simulator configurations: QEMU with KVM acceleration (if available), the behavioral Intel x710 (`i40e`) NIC model, and our simple behavioral switch.

### 1.1. System Configuration
We will start by configuring the system from which we want to create a virtual prototype of. This is the usual way to start a SimBricks script.

Writing this system configurations is about specifying **what** we want to simulate instead of making a choice on how (i.e. which simulator to use) to simulate 

The first step is to create an `System` object. This object contains pointers to all relevant components of the system we want to simulate. Later on we will use those components and decide for each which simulator we want to use.

In [2]:
syst = sys_mod.System()

Now we create a host specification for our client and add it to our system object. In this case we create Linux host that is supposed to have the driver for the IntelI40E nic available. Then we add two disk images. The `DistroDiskImage` is one of the linux images distributed alongside SimBricks that contains the required driver. Note that the `DistroDiskImage` does not take the host it belongs to as a constructor argument. The `LinuxConfigDiskImage` will later on store the actual commands that we want to execute during the simulation on this host.   

In [3]:
# create disk images
distro_disk_image = sys_mod.DistroDiskImage(syst, "base")

# create client
host0 = sys_mod.I40ELinuxHost(syst)
host0.add_disk(distro_disk_image)
host0.add_disk(sys_mod.LinuxConfigDiskImage(syst, host0))

After configuring the client host, we create a specification for an i40e NIC model, and connect it to both the host using a PCIe interface. Under the hood SimBricks system configurations use a notion of device interfaces that are connected through a channel. SImilar to the real world, we further assign an IP address to the NIC that will be made accessible to the host when connecting the NICs interface to the host. 

In [4]:
# create client NIC
nic0 = sys_mod.IntelI40eNIC(syst)
nic0.add_ipv4("10.0.0.1")
host0.connect_pcie_dev(nic0)

Similar to the client, we create a server and attach a nic to the server.

In [5]:
# create server
host1 = sys_mod.I40ELinuxHost(syst)
host1.add_disk(distro_disk_image)
host1.add_disk(sys_mod.LinuxConfigDiskImage(syst, host1))
# create server NIC
nic1 = sys_mod.IntelI40eNIC(syst)
nic1.add_ipv4("10.0.0.2")
host1.connect_pcie_dev(nic1)

After creating and connecting the nic to our client host, we specify the application to run during the simulation. For the client we choose an iperf TCP client and pass it the server IP address to connect to. Further we specify the wait flag on that application. The wait flag is important to tell SimBricks to wait until this application ran through until the simualtion can be stopped and cleaned up (many simulators, for like a network simulator typically run until they are manually stopped when an experiment is executed. To basically tell SimBricks what applications and components we wait for, the wait flag is used).

In [6]:
# set client application
client_app = sys_mod.IperfTCPClient(h=host0, server_ip=nic1._ip)
client_app.wait = True
host0.add_app(client_app)

Again, similar to the client case, we create an iperf server application and assign it to the server host we created before. Note that we do not need to specify the wait flag in this case, as we are interested in the client application to finish, not the server one.

In [7]:
# set server application
server_app = sys_mod.IperfTCPServer(h=host1)
host1.add_app(server_app)

Once we specified the client and server host/NICs we want to simulate, we create a specificatio of a simple switch that we connect to the ethernet interfaces of the previously created NICs to connect those with each other like in a real network.

In [8]:
# create switch and connect NICs to switch
switch = sys_mod.EthSwitch(syst)
switch.connect_eth_peer_if(nic0._eth_if)
switch.connect_eth_peer_if(nic1._eth_if)

And that's it! We have assembled our first SimBricks system specification. We continue buy making a simulation choice.

### 1.2. Simulation Configuration
In the previoius step we configured the system that we want to simulate. After we did this we now have to make a choice on **what simulators** we want to use to simulate this system.

When thinking about this, one realizes that e.g. a linux host might be simualted by e.g. either QEMU or Gem5 whereas a NIC might e.g. be simulated by using a behavioral model or by simulating the actual RTL. The next step is all about making this choice.

The first step is to create a Simulation object:

In [9]:
"""
Simulator Choice
"""
sim = simulation.Simulation(name="My-simple-simulation", system=syst)

In the next step, we go over the system components that we created before (i.e. the two hosts, the two nics and the switch) and create simulator instance. Each of the system components is then added to the simulator that is supposed to simulate that component. 

Depending on whether simulator supports this, you can also choose to add multiple of those components to the same simulator instance, thus causing a single simulator to simulate multiple components at once. This can be very useful. An example of this can be seen in the __[networking-case-study](https://github.com/simbricks/simbricks-examples/tree/orchestration-framework-rework/networking-case-study)__ example within this repo, were we use a single ns3 instance to simulate multiple components (e.g. switches).

In [10]:
host_inst0 = simulation.QemuSim(sim)
host_inst0.add(host0)

host_inst1 = simulation.QemuSim(sim)
host_inst1.add(host1)

nic_inst0 = simulation.I40eNicSim(sim)
nic_inst0.add(nic0)

nic_inst1 = simulation.I40eNicSim(sim)
nic_inst1.add(nic1)

net_inst = simulation.SwitchNet(sim)
net_inst.add(switch)

Alternatively, to make the process of choosing simulators for your components easier, you could use SimBricks helper functions (or define your own helpers using python):

In [11]:
sim = sim_helpers.simple_simulation(
    syst,
    compmap={
        sys_mod.FullSystemHost: simulation.QemuSim,
        sys_mod.IntelI40eNIC: simulation.I40eNicSim,
        sys_mod.EthSwitch: simulation.SwitchNet,
    },
)

If you read the SimBricks paper you know that SimBricks connects simulator instances via shared memory queues to which we refer as `Channel` in the orchestration framework. Those channels can be used in synchronized mode or unsynchronized (default) mode. To e.g. enable to run an experiment synchronized (this is required for accurate time measurements) we need to enable synchronization for those channels:  

In [12]:
# if synchronized set, enable synchronization for all SimBricks channels
synchronized = False
if synchronized:
    sim.enable_synchronization(amount=500, ratio=utils_base.Time.Nanoseconds)

### 1.3 Instantiation

The last thing we need to take care of in order to simulate our virtual prototype is to create an instantiation of it. In our example we create a very simple instantiation and assign the previously created simulation to it.

Additionally we do create a single runtime Fragement that contains all simulators that are supposed to be executed as part of that Fragement, i.e. on the same machine. For this purpose we make use of a SimBricks helper function as well.

In case you plan to distribute the execution of your virtual prototype across multiple machines, you would have to define multiple such Fragments. 

In [13]:
"""
Create an instatiation of your virtual prototype
"""
instantiations = []
instance = inst_helpers.simple_instantiation(sim)
instantiations.append(instance)

And that's it! We have assembled our first SimBricks virtual prototype and we are ready to send it to the server for execution.

## 2 Executing Virtual Prototypes

### 2.1 Via CLI

SimBricks virtual prototype can conviniently be executed via the command line. To do this, the virtual prototyping script must be saved in a .py file (`my-firs-experiment.py` in this case), which has to declare a list Instantiation Configurations called `instantiations`. The Instantiation Configurations in this list along with the respective System- and Simulation-Configurations will then be used for execution.

Here you can see the python script:

In [14]:
# This code just displays nicely formatted contents of my-simple-experiment.py

import IPython


def display_source(code):

    def _jupyterlab_repr_html_(self):
        from pygments import highlight
        from pygments.formatters import HtmlFormatter

        fmt = HtmlFormatter()
        style = '<style>{}\n{}</style>'.format(
            fmt.get_style_defs('.output_html'),
            fmt.get_style_defs('.jp-RenderedHTML')
        )
        return style + highlight(self.data, self._get_lexer(), fmt)

    # Replace _repr_html_ with our own version that adds the 'jp-RenderedHTML' class
    # in addition to 'output_html'.
    IPython.display.Code._repr_html_ = _jupyterlab_repr_html_
    return IPython.display.Code(data=code, language='python3')


with open('my-simple-experiment.py', 'r') as f:
    test2_src = f.read()

display_source(test2_src)

# Copyright 2021 Max Planck Institute for Software Systems, and
# National University of Singapore
#
# Permission is hereby granted, free of charge, to any person obtaining
# a copy of this software and associated documentation files (the
# "Software"), to deal in the Software without restriction, including
# without limitation the rights to use, copy, modify, merge, publish,
# distribute, sublicense, and/or sell copies of the Software, and to
# permit persons to whom the Software is furnished to do so, subject to
# the following conditions:
#
# The above copyright notice and this permission notice shall be
# included in all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
# EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF
# MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT.
# IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY
# CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT,
# TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE
# SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.


from simbricks.orchestration import system
from simbricks.orchestration import simulation
from simbricks.orchestration.helpers import simulation as sim_helpers
from simbricks.orchestration.helpers import instantiation as inst_helpers


"""
This list is used and expected
"""
instantiations = []


"""
System Specification
"""
syst = system.System()

# create disk images
distro_disk_image = system.DistroDiskImage(syst, "base")

# create client
host0 = system.I40ELinuxHost(syst)
host0.add_disk(distro_disk_image)
host0.add_disk(system.LinuxConfigDiskImage(syst, host0))
# create client NIC
nic0 = system.IntelI40eNIC(syst)
nic0.add_ipv4("10.0.0.1")
host0.connect_pcie_dev(nic0)

# create server
host1 = system.I40ELinuxHost(syst)
host1.add_disk(distro_disk_image)
host1.add_disk(system.LinuxConfigDiskImage(syst, host1))
# create server NIC
nic1 = system.IntelI40eNIC(syst)
nic1.add_ipv4("10.0.0.2")
host1.connect_pcie_dev(nic1)

# set client application
client_app = system.IperfTCPClient(h=host0, server_ip=nic1._ip)
client_app.wait = True
host0.add_app(client_app)
# set server application
server_app = system.IperfTCPServer(h=host1)
host1.add_app(server_app)

# create switch and connect NICs to switch
switch = system.EthSwitch(syst)
switch.connect_eth_peer_if(nic0._eth_if)
switch.connect_eth_peer_if(nic1._eth_if)


"""
Simulator Choice
"""
sim = sim_helpers.simple_simulation(
    syst,
    compmap={
        system.FullSystemHost: simulation.QemuSim,
        system.IntelI40eNIC: simulation.I40eNicSim,
        system.EthSwitch: simulation.SwitchNet,
    },
)


"""
Instantiation
"""
instance = inst_helpers.simple_instantiation(sim)
instantiations.append(instance)

To submit the script for execution to the SimBricks server, you can simply run the `simbricks-cli` tool in your terminal:

In [15]:
! simbricks-cli runs submit --follow my-simple-experiment.py

Please visit https://auth.simbricks.io/realms/SimBricks/device in the browser
There, enter the code: UWYX-EFHM
Waiting...
                    Run                     
┏━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ id ┃ instantiation_id ┃ state            ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ 26 │ 26               │ RunState.PENDING │
└────┴──────────────────┴──────────────────┘
[host.QemuSim-28] Formatting sh...0m
'/simbricks/runner-work/run-26/tmp/imgs/hdcopy.28.9.0', fmt=qcow2 
cluster_size=65536 extended_l2=off compression_type=zlib size=42949672960 
backing_file=/simbricks/images/output-base/base backing_fmt=qcow2 
lazy_refcounts=off refcount_bits=16
[host.QemuSim-28] prepare command exited with code 0
[host.QemuSim-24] Formatting ..
'/simbricks/runner-work/run-26/tmp/imgs/hdcopy.24.2.0', fmt=qcow2 
cluster_size=65536 extended_l2=off compression_type=zlib size=42949672960 
backing_file=/simbricks/images/output-base/base backing_fmt=qcow2 
lazy_refcounts=off refcount_bits

### 2.2 Through the python API

SimBricks does also offer a programmatic way to create and submit virtual prototypes to the SimBricks backend in order to schedule their execution on a runner. TODO

In [ ]:
# create and send simulation run to the SimBricks backend
run_id = await opus_base.create_run(instance)

Running your virtual prototype through python directly offers the possibility through retrieve the simulators ourputs line by line in python for parsing and testing. 
This can make retrieving results from the execution of your virtual prototypes easy:  

In [ ]:
# helper function to create and parse the experiment output
async def iperf_throughput() -> None:
    # Regex to match output lines from iperf client
    tp_pat = re.compile(
        r"\[ *\d*\] *([0-9\.]*)- *([0-9\.]*) sec.*Bytes *([0-9\.]*) ([GM])bits.*"
    )
    throughputs = []
    # iterate through host output
    line_gen = opus_base.ConsoleLineGenerator(run_id=run_id, follow=True)
    async for _, line in line_gen.generate_lines():
        m = tp_pat.match(line)
        if not m:
            continue
        if m.group(4) == "G":
            throughputs.append(float(m.group(3)) * 1000)
        elif m.group(4) == "M":
            throughputs.append(float(m.group(3)))

    avg_throughput = sum(throughputs) / len(throughputs)
    print(f"Iperf Throughput : {avg_throughput} Mbps")

await iperf_throughput()